In [6]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

import calendar
import math

# http://images.library.wisc.edu/EcoNatRes/EFacs/Groundwater/BradburyRefine/reference/econatres.bradburyrefine.i0014.pdf

_MONTHDAYS = (31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31)
_LEAP_MONTHDAYS = (31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31)

def sol_dec(day_of_year):
    """
    Calculate solar declination from day of the year.
    Based on FAO equation 24 in Allen et al (1998).
    :param day_of_year: Day of year integer between 1 and 365 or 366).
    :return: solar declination [radians]
    :rtype: float
    """
#    _check_doy(day_of_year)
    return 0.409 * math.sin(((2.0 * math.pi / 365.0) * day_of_year - 1.39))

def sunset_hour_angle(latitude, sol_dec):
    """
    Calculate sunset hour angle (*Ws*) from latitude and solar
    declination.
    Based on FAO equation 25 in Allen et al (1998).
    :param latitude: Latitude [radians]. Note: *latitude* should be negative
        if it in the southern hemisphere, positive if in the northern
        hemisphere.
    :param sol_dec: Solar declination [radians]. Can be calculated using
        ``sol_dec()``.
    :return: Sunset hour angle [radians].
    :rtype: float
    """
#    _check_latitude_rad(latitude)
#    _check_sol_dec_rad(sol_dec)

    cos_sha = -math.tan(latitude) * math.tan(sol_dec)
    # If tmp is >= 1 there is no sunset, i.e. 24 hours of daylight
    # If tmp is <= 1 there is no sunrise, i.e. 24 hours of darkness
    # See http://www.itacanet.org/the-sun-as-a-source-of-energy/
    # part-3-calculating-solar-angles/
    # Domain of acos is -1 <= x <= 1 radians (this is not mentioned in FAO-56!)
    return math.acos(min(max(cos_sha, -1.0), 1.0))

def daylight_hours(sha):
    """
    Calculate daylight hours from sunset hour angle.
    Based on FAO equation 34 in Allen et al (1998).
    :param sha: Sunset hour angle [rad]. Can be calculated using
        ``sunset_hour_angle()``.
    :return: Daylight hours.
    :rtype: float
    """
#    _check_sunset_hour_angle_rad(sha)
    return (24.0 / math.pi) * sha

def thornthwaite(monthly_t, monthly_mean_dlh, year=None):
    """
    Estimate monthly potential evapotranspiration (PET) using the
    Thornthwaite (1948) method.
    Thornthwaite equation:
        *PET* = 1.6 (*L*/12) (*N*/30) (10*Ta* / *I*)***a*
    where:
    * *Ta* is the mean daily air temperature [deg C, if negative use 0] of the
      month being calculated
    * *N* is the number of days in the month being calculated
    * *L* is the mean day length [hours] of the month being calculated
    * *a* = (6.75 x 10-7)*I***3 - (7.71 x 10-5)*I***2 + (1.792 x 10-2)*I* + 0.49239
    * *I* is a heat index which depends on the 12 monthly mean temperatures and
      is calculated as the sum of (*Tai* / 5)**1.514 for each month, where
      Tai is the air temperature for each month in the year
    :param monthly_t: Iterable containing mean daily air temperature for each
        month of the year [deg C].
    :param monthly_mean_dlh: Iterable containing mean daily daylight
        hours for each month of the year (hours]. These can be calculated
        using ``monthly_mean_daylight_hours()``.
    :param year: Year for which PET is required. The only effect of year is
        to change the number of days in February to 29 if it is a leap year.
        If it is left as the default (None), then the year is assumed not to
        be a leap year.
    :return: Estimated monthly potential evaporation of each month of the year
        [mm/month]
    :rtype: List of floats
    """
    if len(monthly_t) != 12:
        raise ValueError(
            'monthly_t should be length 12 but is length {0}.'
            .format(len(monthly_t)))
    if len(monthly_mean_dlh) != 12:
        raise ValueError(
            'monthly_mean_dlh should be length 12 but is length {0}.'
            .format(len(monthly_mean_dlh)))

    if year is None or not calendar.isleap(year):
        month_days = _MONTHDAYS
    else:
        month_days = _LEAP_MONTHDAYS

    # Negative temperatures should be set to zero
    adj_monthly_t = [t * (t >= 0) for t in monthly_t]

    # Calculate the heat index (I)
    I = 0.0
    for Tai in adj_monthly_t:
        if Tai / 5.0 > 0.0:
            I += (Tai / 5.0) ** 1.514

    a = (6.75e-07 * I ** 3) - (7.71e-05 * I ** 2) + (1.792e-02 * I) + 0.49239

    pet = []
    for Ta, L, N in zip(adj_monthly_t, monthly_mean_dlh, month_days):
        # Multiply by 10 to convert cm/month --> mm/month
        pet.append(
            1.6 * (L / 12.0) * (N / 30.0) * ((10.0 * Ta / I) ** a) * 10.0)

    return pet


def monthly_mean_daylight_hours(latitude, year=None):
    """
    Calculate mean daylight hours for each month of the year for a given
    latitude.
    :param latitude: Latitude [radians]
    :param year: Year for the daylight hours are required. The only effect of
        *year* is to change the number of days in Feb to 29 if it is a leap
        year. If left as the default, None, then a normal (non-leap) year is
        assumed.
    :return: Mean daily daylight hours of each month of a year [hours]
    :rtype: List of floats.
    """
#    _check_latitude_rad(latitude)

    if year is None or not calendar.isleap(year):
        month_days = _MONTHDAYS
    else:
        month_days = _LEAP_MONTHDAYS
    monthly_mean_dlh = []
    doy = 1         # Day of the year
    for mdays in month_days:
        dlh = 0.0   # Cumulative daylight hours for the month
        for daynum in range(1, mdays + 1):
            sd = sol_dec(doy)
            sha = sunset_hour_angle(latitude, sd)
            dlh += daylight_hours(sha)
            doy += 1
        # Calc mean daylight hours of the month
        monthly_mean_dlh.append(dlh / mdays)
    return monthly_mean_dlh

In [7]:
from ibmpairs import paw
from datetime import datetime, timedelta
import re, os
from os import path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import requests
import datetime
import zipfile
from zipfile import ZipFile
import glob
import time
from sklearn.linear_model import LinearRegression
from sklearn import datasets, linear_model
import seaborn as sns

iso8601 = '%Y-%m-%dT%H:%M:%SZ'

In [8]:
#Import TMCF coordinates
dfc = pd.read_csv('GIS/TMCF.csv')
dfc

,longitude,latitude,id,tmcf,country,realm
0,-155.589264,-14.966681,0,Ulla Ulla,BOL,Neotropic
1,-105.816653,-17.750014,1,Amboró,BOL,Neotropic
2,-105.000014,-17.266653,2,Carrasco,BOL,Neotropic
3,-104.166653,-20.000014,3,Tariquía,BOL,Neotropic
4,-103.616681,-22.383319,4,Itatiaia,BRA,Neotropic
...,...,...,...,...,...,...
524,149.600014,2.199681,524,Hose Mountains,MYS,Indomalayan
525,150.283042,2.322625,525,Linau Balui Plateau,MYS,Indomalayan
526,150.549597,2.896486,526,Penambo Range,MYS,Indomalayan
527,152.057653,3.306375,527,Dulit Range,MYS,Indomalayan


In [39]:
T_id=49423     #Ground temperature (ERA5)
u10e_id=49420  #10m wind speed East  (ERA5)
u10n_id=49421  #10m wind speed North (ERA5)
TD_id=49422    #Relative humidity (ERA5)
GP_id=49439    #Ground pressure (ERA5)
Rcs_id=49461   #Clear sky (ERA5)
Rs_id=49440    #Solar raditiaton (ERA5)

In [49]:
mET0=[]              #ET0 slope for each coordinate
mPET=[]              #PET slope for each coordinate
for i in range(0, len(dfc):   #Note that eventually you have to loop over all coords in df (1897)
    print(dfc.id[i])
    fname='ts'+str(dfc.id[i])+'.csv'
    dfd= pd.read_csv ('/home/antonio/Documents/TMCF/files/'+fname)
    dT=pd.DataFrame()

    dT['ts']=list(pd.to_datetime(dfd.datetime[dfd.layerId==T_id]))
    dT['T']=list(pd.to_numeric(dfd.value[dfd.layerId==T_id])-273.15)
    dT=dT.set_index(dT['ts']); del dT['ts']

    du10e=pd.DataFrame()
    du10e['ts']=list(pd.to_datetime(dfd.datetime[dfd.layerId==u10e_id]))
    du10e['u10e']=list(pd.to_numeric(dfd.value[dfd.layerId==u10e_id]))
    du10e=du10e.set_index(du10e['ts']); del du10e['ts']

    du10n=pd.DataFrame()
    du10n['ts']=list(pd.to_datetime(dfd.datetime[dfd.layerId==u10n_id]))
    du10n['u10n']=list(pd.to_numeric(dfd.value[dfd.layerId==u10n_id]))
    du10n=du10n.set_index(du10n['ts']); del du10n['ts']

    dTD=pd.DataFrame()
    dTD['ts']=list(pd.to_datetime(dfd.datetime[dfd.layerId==TD_id]))
    dTD['TD']=list(pd.to_numeric(dfd.value[dfd.layerId==TD_id])-273.15)
    dTD=dTD.set_index(dTD['ts']); del dTD['ts']

    dGP=pd.DataFrame()
    dGP['ts']=list(pd.to_datetime(dfd.datetime[dfd.layerId==GP_id]))
    dGP['GP']=list(pd.to_numeric(dfd.value[dfd.layerId==GP_id]))
    dGP=dGP.set_index(dGP['ts']); del dGP['ts']

    dRcs=pd.DataFrame()
    dRcs['ts']=list(pd.to_datetime(dfd.datetime[dfd.layerId==Rcs_id]))
    dRcs['Rcs']=list(pd.to_numeric(dfd.value[dfd.layerId==Rcs_id]))
    dRcs=dRcs.set_index(dRcs['ts']); del dRcs['ts']

    dRs=pd.DataFrame()
    dRs['ts']=list(pd.to_datetime(dfd.datetime[dfd.layerId==Rs_id]))
    dRs['Rs']=list(pd.to_numeric(dfd.value[dfd.layerId==Rs_id]))
    dRs=dRs.set_index(dRs['ts']); del dRs['ts']

    ### REFERENCE EVAPOTRANSPIRATION
    
    dfET0=pd.DataFrame()
    dfET0['T']=dT['T'].resample('1H').mean()
    dfET0['u10e']=du10e['u10e'].resample('1H').mean()
    dfET0['u10n']=du10n['u10n'].resample('1H').mean()
    dfET0['TD']=dTD['TD'].resample('1H').mean()
    dfET0['GP']=dGP['GP'].resample('1H').mean()
    dfET0['Rcs']=dRcs['Rcs'].resample('1H').mean()
    dfET0['Rs']=dRs['Rs'].resample('1H').mean()

    dfET0['u2']=np.sqrt(dfET0['u10e']**2+dfET0['u10n']**2)*0.7479510751679441
    dfET0['RH']=100*np.exp((17.625*dfET0['TD'])/(243.04+dfET0['TD']))/np.exp((17.6258*dfET0['T'])/(243.04+dfET0['T']))

    del dfET0['u10e'],dfET0['u10n'],dfET0['TD']
    del dT,du10e,du10n,dTD,dGP,dRcs,dRs

    dfET0['delta']=4098 * (0.6108 * np.exp((17.27 * dfET0['T']) / (dfET0['T'] + 237.3))) / ((dfET0['T'] + 237.3)**2)
    dfET0['gamma'] = 0.000665 * dfET0['GP'] / 1000.0
    dfET0['DT'] = dfET0['delta'] / (dfET0['delta'] + dfET0['gamma'] * (1 + 0.34 * dfET0['u2'])) # Delta Term (DT) (auxiliary calculation for Radiation Term), order of magnitude 0.5
    dfET0['PT'] = dfET0['gamma'] / (dfET0['delta'] + dfET0['gamma'] * (1 + 0.34 * dfET0['u2'])) # Psi Term (PT) (auxiliary calculation for Wind Term), order of magnitude 0.5
    dfET0['TT'] = (900/24 / (dfET0['T'] + 273)) * dfET0['u2'] #Temperature Term (TT) (auxiliary calculation for Wind Term), order of magnitude 5
    dfET0['es'] = 0.6108 * np.exp(17.27 * dfET0['T'] / (dfET0['T'] + 237.3)) # Mean saturation vapor pressure derived from air temperature (es) [kPa], order of magnitude 1
    dfET0['ea'] = dfET0['RH'] / 100 * dfET0['es'] # Actual vapor pressure (ea) [KPa]
    dfET0['Rs'] = dfET0['Rs'] / 1e6
    dfET0['Rcs']= dfET0['Rcs'] / 1e6
    dfET0['Rs_net'] = (1 - 0.23) * dfET0['Rs'] #Net solar (or net short wave) radiation for albedo a = 0.23 (short grass albedo) [MJ/m2/day], order of magnitude 10
    dfET0['Rl_net'] = 4.903e-9/24 * ((dfET0['T'] + 273.15)**4 + (dfET0['T'] + 273.15)**4) / 2 * (0.34 - 0.14 * np.sqrt(dfET0['ea'])) * (1.35 * dfET0['Rs'] / dfET0['Rcs'] - 0.35) 
    dfET0['R_net'] = dfET0['Rs_net'] - dfET0['Rl_net']  #Net radiation [MJ/m2/day], order of magnitude 10
    dfET0['ET_rad'] = dfET0['DT'] * 0.408 * dfET0['R_net'] #ET radiation term, order of magnitude 0-5 
    dfET0['ET_wind'] = dfET0['PT'] * dfET0['TT']* (dfET0['es'] - dfET0['ea']) #ET wind term, order of magnitude 0-5
    dfET0['ET0_PM'] = dfET0['ET_rad'] + dfET0['ET_wind'] # ET0_PM_calc, order of magnitude 0-10
    #sns.set(rc={'figure.figsize':(30,8)})
    #ax=dfET0['ET0_PM'].plot(linewidth=2.0, marker='.', title='ET0-PM')

    # create an intermittent data frame to fit 
    dffit=pd.DataFrame()
    dffit['ET0_PM']=dfET0['ET0_PM']
    dffit['years']=pd.to_numeric(list(range(len(dfET0))))/(len(dfET0))*23.0
    dffit = clean_dataset(dffit) 
    n=len(dffit)

    x=dffit['years'].values
    y=dffit['ET0_PM'].values
    X = x.reshape(n, 1)
    Y = y.reshape(n, 1)
    regr = linear_model.LinearRegression()
    regr.fit(X,Y)
    mET0.append(regr.coef_[0][0])
    
    
    ### POTENTIAL EVAPOTRANSPIRATION
    
    dffit=pd.DataFrame()
    dffit['T']=dfET0['T']
    dffit['years']=pd.to_numeric(list(range(len(dfET0))))/(len(dfET0))*23.0
    
    dffit=dffit.resample('M').mean()
    
    pet=[]
    for i in range(24):
        t1=pd.to_datetime('1/31/'+str(1997+i))
        t2=pd.to_datetime('12/31/'+str(1997+i))
        monthly_t = list(dffit[t1:t2]['T'])
        lat=float(dfc.latitude[i])*math.pi/180
        monthly_mean_dlh=monthly_mean_daylight_hours(lat)
        pet.append(list(thornthwaite(monthly_t, monthly_mean_dlh,1997+i)))

    from itertools import chain
    pet = list(chain(*pet))
    dffit['PET']=pet
    
    n=len(dffit)
    x=dffit['years'].values
    y=dffit['PET'].values
    
    X = x.reshape(n, 1)
    Y = y.reshape(n, 1)

    #sns.set(rc={'figure.figsize':(30,8)})
    #ax=dfn['ET0_PM'].plot(linewidth=2.0, marker='.', title='ET0-PM')

    regr = linear_model.LinearRegression()
    regr.fit(X,Y)
    mPET.append(regr.coef_[0][0])

348
349


In [50]:
#Export results
file = range(0,346)
frame = pd.DataFrame({"id":file,
                      "mET0":mET0,
                      "mPET":mPET})
frame

ValueError: All arrays must be of the same length

In [46]:
frame.to_csv(r'/home/antonio/Documents/TMCF/ET0_PETb.csv', index = False, header=True)